# Сравнение качества POS-теггеров

1. найти или самим написать **русский и английский тексты (каждый от ста слов)**, в которых будут какие-то трудные или неоднозначные для POS теггинга моменты, **разметить их вручную, объяснить**, какие моменты вы считаете трудными для автоматического посттеггинга и почему. Размечаем только части речи.
*балл за создание, разметку текста и объяснение, почему этот текст подходит для оценки (что в нём сложного). 2 балла*
2. **три POS-теггера для русского** (pymorphy2, mysteam, Natasha) и **3  - для английского** (SpyCy, Flair, NLTK), прогнать текст через каждый из них 
*если вы запустите только 2 теггера из трёх – получите балл, если три из трёх – 2 балла, т. е. суммарно за этот пункт можно получить 4 балла*
3. **оценить accuracy** для каждого теггера. В разных системах имена теггов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции или кода и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции. 
*Этот пункт стоит 2 балла.
Тут вы уже получили 8 баллов*
4. взять лучший теггер для русского языка и с его помощью написать **функцию, которая повысит качество работы программы из первой домашки**. Так, многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Вам надо выделить 3 вида синтаксических групп (к примеру не + какая-то часть речи или NP или сущ.+ наречие или еще что-то), запись которых в словарь, по вашему мнению, улучшила бы качество работы программы и создать такую функцию или функции, которые с помощью любых известных нам средств (chunking и regexp grammar, Natasha syntax parser, код с последнего занятия по SpyCy, etc.) будет выделять эти группы в поданном в нее тексте. 
*Два балла за саму функцию, балл за объяснение того, почему именно эти группы вы взяли*
5. **Встроить эту функцию в программу** из предыдущей домашки и **сравните качества работы** программы с нею и без неё 
*2 бонусных*

## Русский язык

Поскольку у разных теггеров очень разные системы тегов, мы сведём их к максимально общим категориям. Например, выделим А как категорию наречий и прилагательных. Почему не отдельно? Вещи типа "лучше" один теггер считает компаративами, второй прилагательными, третий наречиями. Таким образом, наши классы состоят из A, V, S и ещё нескольких более-иенее универсальных классов. К идеалу привести, конечно же, не получится, так как некоторые категории могут пересекаться непредсказуемо, и чтобы исключить это пересечение, необходимо было бы объединить слишком много.

In [3]:
TEXT1 = '''Отсутствующих людей в классе отмечать было не принято. Считать это злой волей или намеренным вредительством 
    не было причин, по крайней мере тех, которые можно было бы привести в пример, но это все равно казалось странноватым. 
    Достаточно, чтобы задуматься и по-новому оценить по-старому организованный порядок. Несмотря на то, что лишь прыгнув, 
    можно было получить замечание, из-за пропусков завучиха не возникала. Школьный трудовик Иван пытался было возразить, дескать,
    все не по ГОСТу и Минобр будет недобр, но куда там - в принципе, пока никто не косячил, несуразица никого особо 
    не трогала. Де-юре все было чин по чину, де-факто же дисциплина страдала нещадно, и даже разговоры тет-а-тет 
    в директорской приемной не сильно спасали неуклонно падающий псевдоавторитет учительского состава. Ой-ой-ой.
    Зимой было чуть лучше, но за зимой пришла весна, и тут стало интереснее: в апреле появился не только запах ландышей. 
    Апрель запах проблемами.'''

Этот текст неплох для того, чтобы проверять теггеры по нему, так как здесь не каждое место сложное, но их достаточно. К примеру, разметка прилагательных, причастий и субстантивированных существительных. "Директорская" и "приёмная" выглядят одинаково, однако второе хотелось бы видеть размеченным как сууществительное, а первое -- как нечто прилагательноподобное. К тому же, здесь достаточно сокращений (завучиха, трудовик), омонимии (запах), наречий с дефисом внутри (де-юре), новообразованных слов (псевдоавторитет), аббревиатур (Минобр), кратких прилагательных (недобр). 


Проблему в ручной разметке составляют те же самые вводные, а также частицы. Было бы сложно в каких-то случаях точно определить принадлежность слова к части речи. К примеру, как бы мы хотели, чтобы программы анализировали слова типа "чин по чину"? Хотелось бы сказать, конечно, что это наречие, но вряд ли наши морфологизаторы это поймут. То же самое касается вещей типа "куда там" -- определить части речи в этом устоявшемся выражнении сложно. Поскольку, в отличие от авторов преддущего ридинга, не имела возможноти посоветоваться с командой эксперотов, то и разметила всё по разумению своему. Надеюсь, здесь не будет оцениваться качество меня как морфоразметчика. Да простят меня проверяющие за длинные переменные с разбором.

In [224]:
RESULT = [{'слово': 'отсутствующих', 'разбор': 'A'},
 {'слово': 'людей', 'разбор': 'N'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'классе', 'разбор': 'N'},
 {'слово': 'отмечать', 'разбор': 'V'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'принято', 'разбор': 'V'},
 {'слово': 'считать', 'разбор': 'V'},
 {'слово': 'это', 'разбор': 'PART'},
 {'слово': 'злой', 'разбор': 'A'},
 {'слово': 'волей', 'разбор': 'N'},
 {'слово': 'или', 'разбор': 'CONJ'},
 {'слово': 'намеренным', 'разбор': 'A'},
 {'слово': 'вредительством', 'разбор': 'N'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'причин', 'разбор': 'N'},
 {'слово': 'по', 'разбор': 'PREP'},
 {'слово': 'крайней', 'разбор': 'A'},
 {'слово': 'мере', 'разбор': 'N'},
 {'слово': 'тех', 'разбор': 'PRON'},
 {'слово': 'которые', 'разбор': 'A'},
 {'слово': 'можно', 'разбор': 'A'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'бы', 'разбор': 'PART'},
 {'слово': 'привести', 'разбор': 'V'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'пример', 'разбор': 'N'},
 {'слово': 'но', 'разбор': 'CONJ'},
 {'слово': 'это', 'разбор': 'PRON'},
 {'слово': 'все', 'разбор': 'PRON'},
 {'слово': 'равно', 'разбор': 'A'},
 {'слово': 'казалось', 'разбор': 'V'},
 {'слово': 'странноватым', 'разбор': 'A'},
 {'слово': 'достаточно', 'разбор': 'A'},
 {'слово': 'чтобы', 'разбор': 'CONJ'},
 {'слово': 'задуматься', 'разбор': 'V'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'по-новому', 'разбор': 'A'},
 {'слово': 'оценить', 'разбор': 'V'},
 {'слово': 'по-старому', 'разбор': 'A'},
 {'слово': 'организованный', 'разбор': 'A'},
 {'слово': 'порядок', 'разбор': 'N'},
 {'слово': 'несмотря', 'разбор': 'PREP'},
 {'слово': 'на', 'разбор': 'PREP'},
 {'слово': 'то', 'разбор': 'PRON'},
 {'слово': 'что', 'разбор': 'CONJ'},
 {'слово': 'лишь', 'разбор': 'PART'},
 {'слово': 'прыгнув', 'разбор': 'V'},
 {'слово': 'можно', 'разбор': 'A'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'получить', 'разбор': 'V'},
 {'слово': 'замечание', 'разбор': 'N'},
 {'слово': 'из-за', 'разбор': 'PREP'},
 {'слово': 'пропусков', 'разбор': 'N'},
 {'слово': 'завучиха', 'разбор': 'N'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'возникала', 'разбор': 'V'},
 {'слово': 'школьный', 'разбор': 'A'},
 {'слово': 'трудовик', 'разбор': 'N'},
 {'слово': 'иван', 'разбор': 'N'},
 {'слово': 'пытался', 'разбор': 'V'},
 {'слово': 'было', 'разбор': 'PART'},
 {'слово': 'возразить', 'разбор': 'V'},
 {'слово': 'дескать', 'разбор': 'A'},
 {'слово': 'все', 'разбор': 'PRON'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'по', 'разбор': 'PREP'},
 {'слово': 'госту', 'разбор': 'N'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'минобр', 'разбор': 'N'},
 {'слово': 'будет', 'разбор': 'V'},
 {'слово': 'недобр', 'разбор': 'A'},
 {'слово': 'но', 'разбор': 'CONJ'},
 {'слово': 'куда', 'разбор': 'A'},
 {'слово': 'там', 'разбор': 'A'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'принципе', 'разбор': 'N'},
 {'слово': 'пока', 'разбор': 'CONJ'},
 {'слово': 'никто', 'разбор': 'PRON'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'косячил', 'разбор': 'V'},
 {'слово': 'несуразица', 'разбор': 'N'},
 {'слово': 'никого', 'разбор': 'PRON'},
 {'слово': 'особо', 'разбор': 'A'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'трогала', 'разбор': 'V'},
 {'слово': 'де-юре', 'разбор': 'A'},
 {'слово': 'все', 'разбор': 'PRON'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'чин', 'разбор': 'N'},
 {'слово': 'по', 'разбор': 'PREP'},
 {'слово': 'чину', 'разбор': 'N'},
 {'слово': 'де-факто', 'разбор': 'A'},
 {'слово': 'же', 'разбор': 'PART'},
 {'слово': 'дисциплина', 'разбор': 'N'},
 {'слово': 'страдала', 'разбор': 'V'},
 {'слово': 'нещадно', 'разбор': 'A'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'даже', 'разбор': 'PART'},
 {'слово': 'разговоры', 'разбор': 'N'},
 {'слово': 'тет-а-тет', 'разбор': 'A'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'директорской', 'разбор': 'A'},
 {'слово': 'приемной', 'разбор': 'N'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'сильно', 'разбор': 'A'},
 {'слово': 'спасали', 'разбор': 'V'},
 {'слово': 'неуклонно', 'разбор': 'A'},
 {'слово': 'падающий', 'разбор': 'A'},
 {'слово': 'псевдоавторитет', 'разбор': 'N'},
 {'слово': 'учительского', 'разбор': 'A'},
 {'слово': 'состава', 'разбор': 'N'},
 {'слово': 'ой-ой-ой', 'разбор': 'N'},
 {'слово': 'зимой', 'разбор': 'N'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'чуть', 'разбор': 'A'},
 {'слово': 'лучше', 'разбор': 'A'},
 {'слово': 'но', 'разбор': 'CONJ'},
 {'слово': 'за', 'разбор': 'PREP'},
 {'слово': 'зимой', 'разбор': 'N'},
 {'слово': 'пришла', 'разбор': 'V'},
 {'слово': 'весна', 'разбор': 'N'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'тут', 'разбор': 'A'},
 {'слово': 'стало', 'разбор': 'V'},
 {'слово': 'интереснее', 'разбор': 'A'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'апреле', 'разбор': 'N'},
 {'слово': 'появился', 'разбор': 'V'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'только', 'разбор': 'PART'},
 {'слово': 'запах', 'разбор': 'N'},
 {'слово': 'ландышей', 'разбор': 'N'},
 {'слово': 'апрель', 'разбор': 'N'},
 {'слово': 'запах', 'разбор': 'V'},
 {'слово': 'проблемами', 'разбор': 'N'}]

In [2]:
def convert_tags(first_tag):
    if first_tag in {'ADV', 'ADJ', 'A', 'ADJS', 'ADJF', 'COMP', 'PRTF', 'PRTS', 'ADVB', 'NUM', 'NUMR', 'ANUM', 'PRED', 'ADVPRO', 'APRO', 'COM'}:
        last_tag = 'A'
    elif first_tag in {'NOUN', 'S', 'PROPN', 'INTJ'}:
        last_tag = 'N'    
    elif first_tag in {'VERB', 'V', 'INFN', 'GRND', 'AUX'}:
        last_tag = 'V'    
    elif first_tag in {'SCONJ', 'CCONJ', 'CONJ'}:
        last_tag = 'CONJ'    
    elif first_tag in {'ADP', 'PR', 'PREP'}:
        last_tag = 'PREP'    
    elif first_tag in {'PART', 'PRCL'}:
        last_tag = 'PART'
    elif first_tag in {'DET', 'PRON', 'NPRO', 'SPRO'}:
        last_tag = 'PRON'  
    else:
        last_tag = first_tag
        
        
    return last_tag

### Natasha

In [4]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [5]:
segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
#syntax_parser = NewsSyntaxParser(emb)
#ner_tagger = NewsNERTagger(emb)

In [6]:
doc = Doc(TEXT1)
doc.segment(segmenter)
#display(doc.tokens[:5])
#display(doc.sents[:5])

In [7]:
doc.tag_morph(morph_tagger)
#display(doc.tokens[:5])
#doc.sents[0].morph.print()
natasha_result = []
poses = []
for sent in doc.sents:
    for mtoken in sent.morph.tokens:
        t_dict = {}
        t_dict['слово'] = mtoken.text.lower()
        t_dict['разбор'] = convert_tags(mtoken.pos)
        if not t_dict['разбор'] == 'PUNCT':
            natasha_result.append(t_dict)
            poses.append(t_dict['разбор'])

In [228]:
natasha_result[:5]

[{'слово': 'отсутствующих', 'разбор': 'A'},
 {'слово': 'людей', 'разбор': 'N'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'классе', 'разбор': 'N'},
 {'слово': 'отмечать', 'разбор': 'V'}]

### Pymorphy

In [229]:
import nltk
from nltk.tokenize import word_tokenize

tokens = word_tokenize(TEXT1)
print(tokens[:8])

['Отсутствующих', 'людей', 'в', 'классе', 'отмечать', 'было', 'не', 'принято']


In [10]:
%%time
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

pymorphy_result = []

for number, token in enumerate(tokens):
    words = morph.parse(token)
    form = words[0]
    this_word = {}
    this_word['слово'] = form.word
    this_word['разбор'] = convert_tags(form.tag.POS)
    if this_word['разбор'] != None:
        pymorphy_result.append(this_word)

Wall time: 1.17 s


In [230]:
pymorphy_result[:5]

[{'слово': 'отсутствующих', 'разбор': 'A'},
 {'слово': 'людей', 'разбор': 'N'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'классе', 'разбор': 'N'},
 {'слово': 'отмечать', 'разбор': 'V'}]

### Mystem

In [12]:
from pymystem3 import Mystem
m = Mystem()

In [231]:
#%%time

full_stem = m.analyze(TEXT1)
print(full_stem[:5])

[{'analysis': [{'lex': 'отсутствующий', 'wt': 0.008304714066, 'gr': 'A,полн=(пр,мн|род,мн|вин,мн,од)'}], 'text': 'Отсутствующих'}, {'text': ' '}, {'analysis': [{'lex': 'человек', 'wt': 1, 'gr': 'S,муж,од=(вин,мн|род,мн)'}], 'text': 'людей'}, {'text': ' '}, {'analysis': [{'lex': 'в', 'wt': 0.9999917878, 'gr': 'PR='}], 'text': 'в'}]


In [15]:
mystem_result = []

for word in full_stem:
    if word.get('analysis') != None:
        mystem_dict = {}
        mystem_dict['слово'] = word['text'].lower()
        mystem_dict['разбор'] = convert_tags(word['analysis'][0]['gr'].split('=')[0].split(',')[0])
        mystem_result.append(mystem_dict)

In [232]:
mystem_result[:5]

[{'слово': 'отсутствующих', 'разбор': 'A'},
 {'слово': 'людей', 'разбор': 'N'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'классе', 'разбор': 'N'},
 {'слово': 'отмечать', 'разбор': 'V'}]

## Accuracy

В этом разделе функция выдаёт не только accuracy, но и расхождение с эталоном

In [87]:
from sklearn.metrics import accuracy_score

In [22]:
len(natasha_result)

138

In [23]:
len(mystem_result)

138

In [24]:
len(pymorphy_result)

138

Ура, количество токенов одинаковое, всё хорошо!

In [217]:
def detect_acc(any_result, RESULT):
    detected_result = []
    real_result = []
    errors = []
    for i, word in enumerate(any_result):
        if word == RESULT[i]:
            detected_result.append('pos')
        else:
            detected_result.append('neg')
            errors.append([word, RESULT[i]])
        real_result.append('pos')
        
    acc = accuracy_score(detected_result, real_result)
    return acc, errors

In [225]:
detect_acc(natasha_result, RESULT)

(0.9130434782608695,
 [[{'слово': 'которые', 'разбор': 'PRON'},
   {'слово': 'которые', 'разбор': 'A'}],
  [{'слово': 'бы', 'разбор': 'V'}, {'слово': 'бы', 'разбор': 'PART'}],
  [{'слово': 'организованный', 'разбор': 'V'},
   {'слово': 'организованный', 'разбор': 'A'}],
  [{'слово': 'несмотря', 'разбор': 'A'},
   {'слово': 'несмотря', 'разбор': 'PREP'}],
  [{'слово': 'прыгнув', 'разбор': 'N'}, {'слово': 'прыгнув', 'разбор': 'V'}],
  [{'слово': 'было', 'разбор': 'V'}, {'слово': 'было', 'разбор': 'PART'}],
  [{'слово': 'пока', 'разбор': 'A'}, {'слово': 'пока', 'разбор': 'CONJ'}],
  [{'слово': 'несуразица', 'разбор': 'V'},
   {'слово': 'несуразица', 'разбор': 'N'}],
  [{'слово': 'тет-а-тет', 'разбор': 'V'},
   {'слово': 'тет-а-тет', 'разбор': 'A'}],
  [{'слово': 'директорской', 'разбор': 'N'},
   {'слово': 'директорской', 'разбор': 'A'}],
  [{'слово': 'падающий', 'разбор': 'V'}, {'слово': 'падающий', 'разбор': 'A'}],
  [{'слово': 'запах', 'разбор': 'N'}, {'слово': 'запах', 'разбор': 'V'}]

In [226]:
detect_acc(mystem_result, RESULT)

(0.9710144927536232,
 [[{'слово': 'тех', 'разбор': 'A'}, {'слово': 'тех', 'разбор': 'PRON'}],
  [{'слово': 'несмотря', 'разбор': 'A'},
   {'слово': 'несмотря', 'разбор': 'PREP'}],
  [{'слово': 'падающий', 'разбор': 'V'}, {'слово': 'падающий', 'разбор': 'A'}],
  [{'слово': 'запах', 'разбор': 'N'}, {'слово': 'запах', 'разбор': 'V'}]])

In [227]:
detect_acc(pymorphy_result, RESULT)

(0.855072463768116,
 [[{'слово': 'принято', 'разбор': 'A'}, {'слово': 'принято', 'разбор': 'V'}],
  [{'слово': 'волей', 'разбор': 'A'}, {'слово': 'волей', 'разбор': 'N'}],
  [{'слово': 'тех', 'разбор': 'A'}, {'слово': 'тех', 'разбор': 'PRON'}],
  [{'слово': 'это', 'разбор': 'PART'}, {'слово': 'это', 'разбор': 'PRON'}],
  [{'слово': 'все', 'разбор': 'A'}, {'слово': 'все', 'разбор': 'PRON'}],
  [{'слово': 'равно', 'разбор': 'CONJ'}, {'слово': 'равно', 'разбор': 'A'}],
  [{'слово': 'казалось', 'разбор': 'CONJ'},
   {'слово': 'казалось', 'разбор': 'V'}],
  [{'слово': 'то', 'разбор': 'CONJ'}, {'слово': 'то', 'разбор': 'PRON'}],
  [{'слово': 'было', 'разбор': 'V'}, {'слово': 'было', 'разбор': 'PART'}],
  [{'слово': 'дескать', 'разбор': 'CONJ'},
   {'слово': 'дескать', 'разбор': 'A'}],
  [{'слово': 'все', 'разбор': 'A'}, {'слово': 'все', 'разбор': 'PRON'}],
  [{'слово': 'минобр', 'разбор': 'A'}, {'слово': 'минобр', 'разбор': 'N'}],
  [{'слово': 'куда', 'разбор': 'N'}, {'слово': 'куда', 'разбо

## Английский язык

In [195]:
TEXT2 = '''The big set up was set to happen at exactly 7 o'clock - though the clock in question would probably be broken, 
    as would be all clocks in this godforsaken magical town. The citizens were, of course, nonplussed, as in "unperturbed 
    and not ashamed of it", but you as an impostor were very, very cautious - and kinda angry. Life here had its ups and 
    downs, certainly, but the well in your garden was, well...not one of the ups, for sure, more like "lower than lows", 
    whatever that would mean. Without further ado, you were on edge - due to the clocks, the well, and all the little 
    thingies that made you blood burn with barely contained anger. But, hey. As the saying goes, no rose without thorns, 
    right?'''

In [268]:
RESULT2 = [{'слово': 'The', 'разбор': 'DET'},
 {'слово': 'big', 'разбор': 'A'},
 {'слово': 'set', 'разбор': 'N'},
 {'слово': 'up', 'разбор': 'N'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'set', 'разбор': 'V'},
 {'слово': 'to', 'разбор': 'PART'},
 {'слово': 'happen', 'разбор': 'V'},
 {'слово': 'at', 'разбор': 'PREP'},
 {'слово': 'exactly', 'разбор': 'A'},
 {'слово': '7', 'разбор': 'NUM'},
 {'слово': "o'clock", 'разбор': 'N'},
 {'слово': 'though', 'разбор': 'CONJ'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'clock', 'разбор': 'N'},
 {'слово': 'in', 'разбор': 'PREP'},
 {'слово': 'question', 'разбор': 'N'},
 {'слово': 'would', 'разбор': 'V'},
 {'слово': 'probably', 'разбор': 'A'},
 {'слово': 'be', 'разбор': 'V'},
 {'слово': 'broken', 'разбор': 'V'},
 {'слово': 'as', 'разбор': 'CONJ'},
 {'слово': 'would', 'разбор': 'V'},
 {'слово': 'be', 'разбор': 'V'},
 {'слово': 'all', 'разбор': 'DET'},
 {'слово': 'clocks', 'разбор': 'N'},
 {'слово': 'in', 'разбор': 'PREP'},
 {'слово': 'this', 'разбор': 'DET'},
 {'слово': 'godforsaken', 'разбор': 'A'},
 {'слово': 'magical', 'разбор': 'A'},
 {'слово': 'town', 'разбор': 'N'},
 {'слово': 'The', 'разбор': 'DET'},
 {'слово': 'citizens', 'разбор': 'N'},
 {'слово': 'were', 'разбор': 'V'},
 {'слово': 'of', 'разбор': 'A'},
 {'слово': 'course', 'разбор': 'A'},
 {'слово': 'nonplussed', 'разбор': 'V'},
 {'слово': 'as', 'разбор': 'CONJ'},
 {'слово': 'in', 'разбор': 'PREP'},
 {'слово': 'unperturbed', 'разбор': 'A'},
 {'слово': 'and', 'разбор': 'CONJ'},
 {'слово': 'not', 'разбор': 'PART'},
 {'слово': 'ashamed', 'разбор': 'A'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'it', 'разбор': 'PRON'},
 {'слово': 'but', 'разбор': 'CONJ'},
 {'слово': 'you', 'разбор': 'PRON'},
 {'слово': 'as', 'разбор': 'PREP'},
 {'слово': 'an', 'разбор': 'DET'},
 {'слово': 'impostor', 'разбор': 'N'},
 {'слово': 'were', 'разбор': 'V'},
 {'слово': 'very', 'разбор': 'A'},
 {'слово': 'very', 'разбор': 'A'},
 {'слово': 'cautious', 'разбор': 'A'},
 {'слово': 'and', 'разбор': 'CONJ'},
 {'слово': 'kinda', 'разбор': 'A'},
 {'слово': 'angry', 'разбор': 'A'},
 {'слово': 'Life', 'разбор': 'N'},
 {'слово': 'here', 'разбор': 'A'},
 {'слово': 'had', 'разбор': 'V'},
 {'слово': 'its', 'разбор': 'PRON'},
 {'слово': 'ups', 'разбор': 'N'},
 {'слово': 'and', 'разбор': 'CONJ'},
 {'слово': 'downs', 'разбор': 'N'},
 {'слово': 'certainly', 'разбор': 'A'},
 {'слово': 'but', 'разбор': 'CONJ'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'well', 'разбор': 'N'},
 {'слово': 'in', 'разбор': 'PREP'},
 {'слово': 'your', 'разбор': 'PRON'},
 {'слово': 'garden', 'разбор': 'N'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'well', 'разбор': 'INTJ'},
 {'слово': 'not', 'разбор': 'PART'},
 {'слово': 'one', 'разбор': 'NUM'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'ups', 'разбор': 'N'},
 {'слово': 'for', 'разбор': 'PREP'},
 {'слово': 'sure', 'разбор': 'A'},
 {'слово': 'more', 'разбор': 'A'},
 {'слово': 'like', 'разбор': 'PREP'},
 {'слово': 'lower', 'разбор': 'A'},
 {'слово': 'than', 'разбор': 'CONJ'},
 {'слово': 'lows', 'разбор': 'N'},
 {'слово': 'whatever', 'разбор': 'PRON'},
 {'слово': 'that', 'разбор': 'PRON'},
 {'слово': 'would', 'разбор': 'V'},
 {'слово': 'mean', 'разбор': 'V'},
 {'слово': 'Without', 'разбор': 'PREP'},
 {'слово': 'further', 'разбор': 'A'},
 {'слово': 'ado', 'разбор': 'N'},
 {'слово': 'you', 'разбор': 'PRON'},
 {'слово': 'were', 'разбор': 'V'},
 {'слово': 'on', 'разбор': 'PREP'},
 {'слово': 'edge', 'разбор': 'N'},
 {'слово': 'due', 'разбор': 'PREP'},
 {'слово': 'to', 'разбор': 'PREP'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'clocks', 'разбор': 'N'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'well', 'разбор': 'A'},
 {'слово': 'and', 'разбор': 'CONJ'},
 {'слово': 'all', 'разбор': 'DET'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'little', 'разбор': 'A'},
 {'слово': 'thingies', 'разбор': 'N'},
 {'слово': 'that', 'разбор': 'CONJ'},
 {'слово': 'made', 'разбор': 'V'},
 {'слово': 'you', 'разбор': 'PRON'},
 {'слово': 'blood', 'разбор': 'N'},
 {'слово': 'burn', 'разбор': 'V'},
 {'слово': 'with', 'разбор': 'PREP'},
 {'слово': 'barely', 'разбор': 'A'},
 {'слово': 'contained', 'разбор': 'V'},
 {'слово': 'anger', 'разбор': 'N'},
 {'слово': 'But', 'разбор': 'CONJ'},
 {'слово': 'hey', 'разбор': 'INTJ'},
 {'слово': 'As', 'разбор': 'CONJ'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'saying', 'разбор': 'N'},
 {'слово': 'goes', 'разбор': 'V'},
 {'слово': 'no', 'разбор': 'DET'},
 {'слово': 'rose', 'разбор': 'N'},
 {'слово': 'without', 'разбор': 'PREP'},
 {'слово': 'thorns', 'разбор': 'N'},
 {'слово': 'right', 'разбор': 'A'}]

In [244]:
def convert_eng_tags(first_tag):
    if first_tag in {'ADJ', 'ADV', 'RB', 'RBR', 'RBS', 'JJ', 'JJR', 'JJS', }:
        last_tag = 'A'
    elif first_tag in {'PROPN', 'NOUN', 'NN', 'NNS', 'NNP', 'NNPS'}:
        last_tag = 'N'    
    elif first_tag in  {'VERB', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'MD', 'AUX'}:
        last_tag = 'V'    
    elif first_tag in {'SCONJ', 'CCONJ', 'CONJ', 'CC', 'PCONJ'}:
        last_tag = 'CONJ'    
    elif first_tag in {'ADP', 'TO', 'IN'}:
        last_tag = 'PREP'    
    elif first_tag in {'PART', 'POS', 'RP'}:
        last_tag = 'PART'
    elif first_tag in {'PRON', 'WDT', 'WP', 'WP$', 'WRB', 'PRP', 'PRP$', 'EX'}:
        last_tag = 'PRON'  
    elif first_tag in {'DT', 'DET', 'PDT'}:
        last_tag = 'DET'  
    elif first_tag in {'NUM', 'CD'}:
        last_tag = 'NUM'  
    elif first_tag in {'INTJ', 'UH'}:
        last_tag = 'INTJ'  
    else:
        last_tag = 'PUNCT'
        
        
    return last_tag

### Spacy

In [91]:
import spacy

nlp = spacy.load("en_core_web_sm")


-- Sentence 0 --
The	DET
big	ADJ
set	NOUN
up	ADP
was	AUX
set	VERB
to	PART
happen	VERB
at	ADP
exactly	ADV
7	NUM
o’clock	NOUN
-	PUNCT
though	SCONJ
the	DET
clock	NOUN
in	ADP
question	NOUN
would	VERB
probably	ADV
be	AUX
broken	VERB
,	PUNCT

    	SPACE
as	SCONJ
would	VERB
be	AUX
all	DET
clocks	NOUN
in	ADP
this	DET
godforsaken	ADJ
magical	ADJ
town	NOUN
.	PUNCT

-- Sentence 1 --
The	DET
citizens	NOUN
were	AUX
,	PUNCT
of	ADP
course	NOUN
,	PUNCT
nonplussed	VERB
,	PUNCT
as	SCONJ
in	ADP
“	PUNCT
unperturbed	ADJ

    	SPACE
and	CCONJ
not	PART
ashamed	ADJ
of	ADP
it	PRON
”	PUNCT
,	PUNCT
but	CCONJ
you	PRON
as	SCONJ
an	DET
impostor	NOUN
were	AUX
very	ADV
,	PUNCT
very	ADV
cautious	ADJ
-	PUNCT
and	CCONJ
kinda	ADV
angry	ADJ
.	PUNCT

-- Sentence 2 --
Life	NOUN
here	ADV
had	AUX
its	DET
ups	NOUN
and	CCONJ

    	SPACE
downs	NOUN
,	PUNCT
certainly	ADV
,	PUNCT
but	CCONJ
the	DET
well	NOUN
in	ADP
your	DET
garden	NOUN
was	AUX
,	PUNCT
well	INTJ
...	PUNCT
not	PART
one	NUM
of	ADP
the	DET
ups	NOUN
,	PUNCT
for	ADP
sur

In [245]:
doc = nlp(TEXT2)
spacy_result = []
for sent in doc.sents:
    for t in sent:
        word = {}
        word['слово'] = t.text
        word['разбор'] = convert_eng_tags(t.pos_)
        if word['разбор'] != 'PUNCT':
            spacy_result.append(word)

In [246]:
len(spacy_result)

127

### Flair

Осторожно, долгая загрузка. Зато теги хороши!

In [109]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load('pos-multi')

2020-10-18 20:01:43,291 https://nlp.informatik.hu-berlin.de/resources/models/multi-pos/pos-multi-v0.1.pt not found in cache, downloading to C:\Users\M\AppData\Local\Temp\tmp35vs92ee


100%|████████████████████████████████████████████████████████████████| 314055714/314055714 [1:23:39<00:00, 62565.23B/s]

2020-10-18 21:25:23,390 copying C:\Users\M\AppData\Local\Temp\tmp35vs92ee to cache at C:\Users\M\.flair\models\pos-multi-v0.1.pt


2020-10-18 21:25:27,162 removing temp file C:\Users\M\AppData\Local\Temp\tmp35vs92ee
2020-10-18 21:25:27,298 loading file C:\Users\M\.flair\models\pos-multi-v0.1.pt


In [247]:
sentence = Sentence(TEXT2)
tagger.predict(sentence)

#print(sentence)
f_result = sentence.to_tagged_string()

In [213]:
#f_result

In [248]:
flair_result = []
for word in f_result.split('> '):
    word = word.split(' <')
    new_word = {}
    new_word['слово'] = word[0]
    new_word['разбор'] = convert_eng_tags(word[1].strip('>'))
    if new_word['разбор'] != 'PUNCT':
        flair_result.append(new_word)

In [249]:
len(flair_result)

127

### NLTK

In [111]:
import nltk
from nltk.tokenize import word_tokenize

In [115]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\M\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [258]:
tokens = word_tokenize(TEXT2)

print(tokens[0:10])

['The', 'big', 'set', 'up', 'was', 'set', 'to', 'happen', 'at', 'exactly']


In [259]:
nltk_result = []
for word in nltk.pos_tag(tokens):
    new_word = {}
    new_word['слово'] = word[0]
    new_word['разбор'] = convert_eng_tags(word[1])
    if new_word['разбор'] != 'PUNCT':
        nltk_result.append(new_word)

In [261]:
len(nltk_result)

127

## Accuracy для английского

In [269]:
detect_acc(spacy_result, RESULT2)

(0.9133858267716536,
 [[{'слово': 'up', 'разбор': 'PREP'}, {'слово': 'up', 'разбор': 'N'}],
  [{'слово': 'of', 'разбор': 'PREP'}, {'слово': 'of', 'разбор': 'A'}],
  [{'слово': 'course', 'разбор': 'N'}, {'слово': 'course', 'разбор': 'A'}],
  [{'слово': 'as', 'разбор': 'CONJ'}, {'слово': 'as', 'разбор': 'PREP'}],
  [{'слово': 'its', 'разбор': 'DET'}, {'слово': 'its', 'разбор': 'PRON'}],
  [{'слово': 'your', 'разбор': 'DET'}, {'слово': 'your', 'разбор': 'PRON'}],
  [{'слово': 'like', 'разбор': 'CONJ'}, {'слово': 'like', 'разбор': 'PREP'}],
  [{'слово': 'whatever', 'разбор': 'DET'},
   {'слово': 'whatever', 'разбор': 'PRON'}],
  [{'слово': 'that', 'разбор': 'DET'}, {'слово': 'that', 'разбор': 'PRON'}],
  [{'слово': 'that', 'разбор': 'DET'}, {'слово': 'that', 'разбор': 'CONJ'}],
  [{'слово': 'rose', 'разбор': 'V'}, {'слово': 'rose', 'разбор': 'N'}]])

In [270]:
detect_acc(flair_result, RESULT2)

(0.9606299212598425,
 [[{'слово': 'well', 'разбор': 'A'}, {'слово': 'well', 'разбор': 'INTJ'}],
  [{'слово': 'not', 'разбор': 'A'}, {'слово': 'not', 'разбор': 'PART'}],
  [{'слово': 'than', 'разбор': 'PREP'}, {'слово': 'than', 'разбор': 'CONJ'}],
  [{'слово': 'that', 'разбор': 'PRON'}, {'слово': 'that', 'разбор': 'CONJ'}],
  [{'слово': 'right', 'разбор': 'INTJ'}, {'слово': 'right', 'разбор': 'A'}]])

In [266]:
detect_acc(nltk_result, RESULT2)

(0.8110236220472441,
 [[{'слово': 'up', 'разбор': 'PART'}, {'слово': 'up', 'разбор': 'N'}],
  [{'слово': 'to', 'разбор': 'PREP'}, {'слово': 'to', 'разбор': 'PART'}],
  [{'слово': "o'clock", 'разбор': 'A'}, {'слово': "o'clock", 'разбор': 'N'}],
  [{'слово': 'though', 'разбор': 'PREP'},
   {'слово': 'though', 'разбор': 'CONJ'}],
  [{'слово': 'as', 'разбор': 'PREP'}, {'слово': 'as', 'разбор': 'CONJ'}],
  [{'слово': 'of', 'разбор': 'PREP'}, {'слово': 'of', 'разбор': 'A'}],
  [{'слово': 'course', 'разбор': 'N'}, {'слово': 'course', 'разбор': 'A'}],
  [{'слово': 'nonplussed', 'разбор': 'A'},
   {'слово': 'nonplussed', 'разбор': 'V'}],
  [{'слово': 'as', 'разбор': 'PREP'}, {'слово': 'as', 'разбор': 'CONJ'}],
  [{'слово': 'not', 'разбор': 'A'}, {'слово': 'not', 'разбор': 'PART'}],
  [{'слово': 'ashamed', 'разбор': 'V'}, {'слово': 'ashamed', 'разбор': 'A'}],
  [{'слово': 'kinda', 'разбор': 'V'}, {'слово': 'kinda', 'разбор': 'A'}],
  [{'слово': 'well', 'разбор': 'A'}, {'слово': 'well', 'разбор':

## Итоги

Итак, у нас получилось что-то, что показывает, что те библиотеки, которые больше ориентируются на контекст, точнее, но и они не идеальны.